## Probelm Statement:

To build a classification model using word embeddings to classify email messages as Spam or Ham.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Required Libraries:

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding
from warnings import filterwarnings
filterwarnings('ignore')

## Loading the data:

In [ ]:
data_spam=pd.read_csv("../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")
data_spam.head(10)

In [ ]:
# Replace Spam with 1 and Ham with 0
data_spam['Category']=data_spam['Category'].replace({'ham':0,'spam':1})
data_spam.head()

In [ ]:
data_spam.shape    # 5572 rows and 2 columns available

## Text Pre-processing:

In [ ]:
# Original sentence before text pre-processing
data_spam['Message'][2]

In [ ]:
stop_words=stopwords.words('english')
lemma=WordNetLemmatizer()

sentence=data_spam['Message']

new_sentence=[]
for i in range(len(sentence)):
    
    # excluding the irrelavent characters
    word=re.sub("[^a-zA-Z\.\!]"," ",sentence[i])
    
    #Lower casing all words
    word=word.lower()
    
    #Splitting each sentence into words
    word=word.split()
    
    #Performing lemmatization to retain the root word out of all the inflected words. Also eliminating stopwords
    word=[lemma.lemmatize(w) for w in word if w not in stop_words]
    word=" ".join(word)
    new_sentence.append(word)

#Sentence after pre-processing
new_sentence[2]

In [ ]:
data_spam['New_Message']=new_sentence
data_spam.head()

## Building a Word Embedding Matrix:

In [ ]:
# Loading a pre-trained word2vec model 
embeddings=KeyedVectors.load_word2vec_format('../input/word2vec-google/GoogleNews-vectors-negative300.bin',binary=True)


In [ ]:
# Splitting data into train and test
train_X,test_X,train_y,test_y=train_test_split(data_spam['New_Message'],data_spam['Category'],test_size=0.3,random_state=10)

In [ ]:
# Initialising the Tokenizer
tokenizer=Tokenizer()

# Fitting the tokenizer on train_x
tokenizer.fit_on_texts(train_X)

# Number of words 
vocab_size=len(tokenizer.word_index)+1

#Converting the words to sequence of their corresponding index
train_X_seq=tokenizer.texts_to_sequences(train_X)
test_X_seq=tokenizer.texts_to_sequences(test_X)

In [ ]:

#Obtaining the length of each sequence list inoder to find the maximum length which is to be used for padding the train and test
len_doc=[]
for doc in train_X_seq:
    doc_size=len(doc)
    len_doc.append(doc_size)

print("Length of First 10 sequence:",len_doc[:10])
print("Maximum length of sequence:",max(len_doc))

sns.boxplot(len_doc,color='green')
plt.title("Distribution of length of sequence list")
plt.show()

In [ ]:
#Padding each list of sequence to ensure each of them have same dimension.Considering max length as 97

train_X_pad=pad_sequences(train_X_seq,maxlen=97,padding='post')
test_X_pad=pad_sequences(test_X_seq,maxlen=97,padding='post')

In [ ]:
# Building the weight matrix 
embeddings_mat=np.zeros((vocab_size,300))
words_available=[]
words_not_available=[]

for w,wid in tokenizer.word_index.items():
    if w in embeddings:
        embeddings_mat[wid]=embeddings[w]
        words_available.append(w)
        
    else:
        words_not_available.append(w)

In [ ]:
# Amount of words not available in the pre-trained embeddings model
len(words_not_available)/vocab_size *100

## Building a Classification model:

In [ ]:
# Initialising the model
model=Sequential()

# Since we already have the weights matrix, there is not need to further train the weights and hence setting trainable parameter as False
model.add(Embedding(vocab_size,300,weights=[embeddings_mat],input_length=97,trainable=False))

# Flattening the layers
model.add(Dense(units=16,activation='relu'))

#Output Layer
model.add(Dense(units=1,activation='sigmoid'))

## Compiling the model:

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Fitting the model on Train data:

In [ ]:
my_model=model.fit(x=train_X_pad,y=train_y,batch_size=32,epochs=50,validation_data=(test_X_pad,test_y))

## Comparing Performance:

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(my_model.history['loss'],label='train_loss')
plt.plot(my_model.history['val_loss'],label='test_loss')
plt.title("Loss")
plt.legend()
plt.suptitle("Performance evaluation")

plt.subplot(122)
plt.plot(my_model.history['accuracy'],label='train_accuracy')
plt.plot(my_model.history['val_accuracy'],label='test_accuracy')
plt.title("Accuracy")
plt.legend()
plt.show()

## Conclusion:

- A spam classifier was built using  a pre-trained word embedding model

- The model gave an accuracy of 87.4% on Train data and 87.8% on Test data.

- The model gave a binaray cross-entropy loss of 0.37 on Train data and 0.36 on test data.

- As per the scores, the model built exhibited exceptionally low over-fitting.